### Try-it 8.1: The "Best" Model

This module was all about regression and using Python's scikitlearn library to build regression models.  Below, a dataset related to real estate prices in California is given. While many of the assignments you have built and evaluated different models, it is important to spend some time interpreting the resulting "best" model.  


Your goal is to build a regression model to predict the price of a house in California.  After doing so, you are to *interpret* the model.  There are many strategies for doing so, including some built in methods from scikitlearn.  One example is `permutation_importance`.  Permutation feature importance is a strategy for inspecting a model and its features importance.  

Take a look at the user guide for `permutation_importance` [here](https://scikit-learn.org/stable/modules/permutation_importance.html).  Use  the `sklearn.inspection` modules implementation of `permutation_importance` to investigate the importance of different features to your regression models.  Share these results on the discussion board.

In [24]:
import pandas as pd
from sklearn.inspection import permutation_importance
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.compose import make_column_transformer, make_column_selector



In [25]:
import numpy as np
import matplotlib.pyplot as plt

In [26]:
cali = pd.read_csv('data/housing.csv')

In [27]:
cali.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [28]:
cali.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


## Data Cleaning

- It looks like feature total_bedrooms has missing value, roughly around 207 data points.
- Since we are observing the median value of both income and house value, for the sake of consistency, let's perform a median imputation.
- Sklearn has a module called SimpleImputer

In [29]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')

cali['total_bedrooms'] = imputer.fit_transform(cali[['total_bedrooms']])

# let's check the missing value
cali['total_bedrooms'].isnull().sum()

cali.isnull().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

Great, now that we have no more missing values.

Let's decide what are we predicting using what features do we need for the model. Also, we need to decide how much data we are going to use for training. Then once we get models ready for prediction on testing data, we need to compare which model best predicting the unseen data.

## Baseline Model

In [30]:
# spliting X and y dataset
X = cali.drop('median_house_value', axis=1)
y = cali['median_house_value']

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
# we want to observe how much error between the mean and the ground truth testing data
baseline_train = np.ones(shape=y_train.shape)*y_train.mean()
baseline_test = np.ones(shape=y_test.shape)*y_test.mean()

mse_baseline_train = mean_squared_error(y_train, baseline_train)
mse_baseline_test = mean_squared_error(y_test, baseline_test)

print(f'Baseline for testing data: {mse_baseline_test}')

Baseline for testing data: 13104089782.408998


The different in two function np.ones() and np.full_like() yield slight different number due to floating-point arithmetic precision or rounding differences

In [33]:
# the categories feature is ocean_proximity
# let create one hot coding feature model

ocean_proximity_train = X_train[['ocean_proximity']]
ocean_proximity_test = X_test[['ocean_proximity']]

In [34]:
ohe = OneHotEncoder(sparse_output=False, drop='if_binary')

In [35]:
model_ohe_train = ohe.fit_transform(ocean_proximity_train)
model_ohe_test = ohe.transform(ocean_proximity_test)


In [36]:
model_ohe = LinearRegression().fit(model_ohe_train, y_train)

In [37]:
model_ohe_preds = model_ohe.predict(model_ohe_test)
mse_test = mean_squared_error(model_ohe_preds, y_test)

print(f'One Hot Coding for testing data: {mse_test}')

One Hot Coding for testing data: 9877034768.580427


In [38]:
# lastly, we want to model both with features one hot coding from categorical variable
# and polynomial features of numerical features

test_mses = []

for i in range(1, 6):
    poly_ordinal_ohe = make_column_transformer((PolynomialFeatures(degree = i), make_column_selector(dtype_include=np.number)),
                                               (OneHotEncoder(drop = 'if_binary'), ['ocean_proximity']))
    pipe = Pipeline([('transformer', poly_ordinal_ohe), ('linreg', LinearRegression())])

    pipe.fit(X_train, y_train)
    p2 = pipe.predict(X_test)

    test_mses.append(mean_squared_error(y_test, p2))

print(test_mses)

[4908476721.156556, 4496494023.057793, 10423035059.009874, 29334469524930.312, 6443795526368757.0]


All these result yield the same number as the module. That is a good sign that my code work well and cleaner.

As we observe, the model Degree 2 + One Hot Coding: MSE = 4,496,494,023.057793 (Best Performance)

Please check my class module that perform the step above in one function: ModelComparison()

### Data Splitting + Model Fitting + Model Comparison Integration

Since we have to create so many function and messy, I decided to create a class that would perform all these task simultaneuous for cleaner code

In [39]:
from ModelComparison import ModelComparison

In [40]:

# Define target column and feature columns
target_column = 'median_house_value'
categorical_columns = ['ocean_proximity']
numerical_columns = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 
                     'total_bedrooms', 'population', 'households', 'median_income']

# Instantiate the ModelComparison class
model_comparison = ModelComparison(dataframe=cali, 
                                   target_column=target_column, 
                                   categorical_columns=categorical_columns, 
                                   numerical_columns=numerical_columns,
                                   poly_degree=3)

# Fit the models and calculate MSE
model_comparison.fit()

# Retrieve and print the MSE for each model
model_performance = model_comparison.get_model_performance()
print(model_performance)


{'Baseline Model': 13106960720.039366, 'One-Hot Model': 9876284291.152872, 'Poly + One-Hot Model': 10423035059.009874}


In the class, I split the data 20% test size and random value=42

In [41]:

# Define target and feature columns
target_column = 'median_house_value'
categorical_columns = ['ocean_proximity']
numerical_columns = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 
                     'total_bedrooms', 'population', 'households', 'median_income']

# Range of polynomial degrees you want to test
poly_degrees = range(1, 6)  # For example, degrees 1 through 4

# Dictionary to store MSEs for each degree
degree_mse = {}

# Loop over each degree, instantiate the class, fit models, and store performance
for degree in poly_degrees:
    model_comparison = ModelComparison(dataframe=cali, 
                                       target_column=target_column, 
                                       categorical_columns=categorical_columns, 
                                       numerical_columns=numerical_columns,
                                       poly_degree=degree)
    model_comparison.fit()
    model_performance = model_comparison.get_model_performance()
    
    # the MSE of the Poly + One-Hot Model
    mse_for_degree = model_performance.get('Poly + One-Hot Model')
    degree_mse[degree] = mse_for_degree

# Print MSEs for each degree
for degree, mse in degree_mse.items():
    print(f"Degree {degree}: MSE = {mse}")


Degree 1: MSE = 4908476721.156556
Degree 2: MSE = 4496494023.057793
Degree 3: MSE = 10423035059.009874
Degree 4: MSE = 29334469524930.312
Degree 5: MSE = 6443795526368757.0


In [42]:
best_degree = min(degree_mse, key=degree_mse.get)  # Finds the degree with the lowest MSE
best_mse = degree_mse[best_degree]
print(f"Best degree: {best_degree} with MSE: {best_mse}")


Best degree: 2 with MSE: 4496494023.057793


In [43]:
best_model_comparison = ModelComparison(dataframe=cali, 
                                        target_column=target_column, 
                                        categorical_columns=categorical_columns, 
                                        numerical_columns=numerical_columns,
                                        poly_degree=best_degree)
best_model_comparison.fit()


In [44]:
best_model_name, best_mse, best_model = best_model_comparison.get_best_model()
print(f"Best model: {best_model_name} with MSE: {best_mse}")

Best model: Poly + One-Hot Model with MSE: 4496494023.057793


In [45]:
best_model.predict(X_test)

array([ 79111.3992028 ,  87206.94064916, 272325.57262656, ...,
       464827.11268383, 106487.99352048, 187313.3832815 ])

## Interpretation:

In [46]:
from sklearn.inspection import permutation_importance

result = permutation_importance(best_model, X_test, y_test, n_repeats=10, random_state=42)

# Organizing and displaying feature importances
importances = result.importances_mean
features = X_test.columns
feature_importance_dict = {feature: importance for feature, importance in zip(features, importances)}
sorted_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

print("Feature Importance (Mean Decrease in Performance):")
for feature, importance in sorted_importance:
    print(f"{feature}: {importance}")


Feature Importance (Mean Decrease in Performance):
latitude: 1.920732685581489
longitude: 1.842702395403527
total_rooms: 1.6949432056070677
total_bedrooms: 1.3165710442403553
households: 0.9845214833281576
median_income: 0.8980990528934527
population: 0.6366297349437874
housing_median_age: 0.07165923245163114
ocean_proximity: 0.023049872401397442
